<a href="https://colab.research.google.com/github/Aidzillafont/-Machine-Learning-Model-Selection-and-Optimization-for-Classification-Task/blob/main/Machine_Learning_Model_Selection_and_Optimization_for_Classification_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Objective

The Boolean satisfiability (SAT) problem consists in determining whether a Boolean formula F is satisfiable or not. F is represented by a pair (X, C), where X is a set of Boolean variables and C is a set of clauses in Conjunctive Normal Form (CNF). Each clause is a disjunction of literals (a variable or its negation). This problem is one of the most widely studied combinatorial problems in computer science. It is the classic NP-complete problem. Over the past number of decades, a significant amount of research work has focused on solving SAT problems with both complete and incomplete solvers.

Recent advances in supervised learning have provided powerful techniques for classifying problems. In this project, we see the SAT problem as a classification problem. Given a Boolean formula (represented by a vector of features), we are asked to predict if it is satisfiable or not.

In this project, we represent SAT problems with a vector of 327 features with general information about the problem, e.g., number of variables, number of clauses, fraction of horn clauses in the problem, etc. There is no need to understand the features to be able to complete the assignment.

The dataset is available at:
https://github.com/andvise/DataAnalyticsDatasets/blob/main/dm_assignment2/sat_dataset_train.csv

This is original unpublished data.

## Data Preparation

In [ ]:
import pandas as pd

df = pd.read_csv("https://github.com/andvise/DataAnalyticsDatasets/blob/6d5738101d173b97c565f143f945dedb9c42a400/dm_assignment2/sat_dataset_train.csv?raw=true")
df.head()

,c,v,clauses_vars_ratio,vars_clauses_ratio,vcg_var_mean,vcg_var_coeff,vcg_var_min,vcg_var_max,vcg_var_entropy,vcg_clause_mean,...,rwh_0_max,rwh_1_mean,rwh_1_coeff,rwh_1_min,rwh_1_max,rwh_2_mean,rwh_2_coeff,rwh_2_min,rwh_2_max,target
0,420,10,42.000000,0.023810,0.600000,0.000000,0.600000,0.600000,0.000000,0.600000,...,78750.0,0.000008,0.0,7.875000e-06,0.000008,2.385082e-21,0.0,2.385082e-21,2.385082e-21,1
1,230,20,11.500000,0.086957,0.137826,0.089281,0.117391,0.160870,2.180946,0.137826,...,6646875.0,17433.722184,1.0,2.981244e-12,34867.444369,1.727721e+04,1.0,1.358551e-53,3.455442e+04,0
2,240,16,15.000000,0.066667,0.300000,0.000000,0.300000,0.300000,0.000000,0.300000,...,500000.0,1525.878932,0.0,1.525879e+03,1525.878932,1.525879e+03,0.0,1.525879e+03,1.525879e+03,1
3,424,30,14.133333,0.070755,0.226415,0.485913,0.056604,0.452830,2.220088,0.226415,...,87500.0,0.000122,1.0,6.535723e-14,0.000245,8.218628e-07,1.0,1.499676e-61,1.643726e-06,0
4,162,19,8.526316,0.117284,0.139701,0.121821,0.111111,0.185185,1.940843,0.139701,...,5859400.0,16591.494310,1.0,6.912726e-42,33182.988621,1.665903e+04,1.0,0.000000e+00,3.331807e+04,1


In [ ]:
df.dtypes

c                       int64
v                       int64
clauses_vars_ratio    float64
vars_clauses_ratio    float64
vcg_var_mean          float64
                       ...   
rwh_2_mean            float64
rwh_2_coeff           float64
rwh_2_min             float64
rwh_2_max             float64
target                  int64
Length: 328, dtype: object

In [ ]:
df['target'].value_counts()

1    976
0    953
Name: target, dtype: int64

### Feature Selection and Sparsity Evaluation

1. Select the features by considering all columns except the target column, and assign the target column to the label variable.
2. Evaluate the sparsity of the entire feature set. Since it contains 31% zeros, the dataset is not considered sparse.
3. Create a train-test split with a random state of 6405 and a test size of 30%.


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

features = list(df.columns)
features.pop()
labels = ['target']

X,y = df[features], df[labels]

print('Ratio of sparse points to total data points in our feature set is: ', (X.to_numpy() == 0).mean())

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.30, random_state=6405)


Ratio of sparse points to total data points in our feature set is:  0.31475800711179597


### Replace infs with nans Custom Transformer
Since there is a number of infs in our feature set we will replace these with nans and then impute these values later

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# a custom transformation for my pipeline changes inf and -inf to np.nan for simple imputer
class RemoveInfs(BaseEstimator, TransformerMixin) : 
     def __init__(self) : 
          pass

     def fit(self, X, y=None) : 
          return self

     def transform(self, X) : 
          X = X.replace([np.inf, -np.inf], np.nan)
          return X

# Tasks


## Basic models and evaluation

Using Scikit-learn, train and evaluate K-NN and decision tree classifiers using 70% of the dataset from training and 30% for testing. We just want to get an idea of the dataset. Compare the results of both classifiers.

### Pipeline Overview

1. The fundamental pipeline incorporates a custom transformation, `RemoveInf()`, which replaces infinite values with NaNs.
2. Next, the `SimpleImputer()` is used to replace NaNs with the mean value of the corresponding feature.
3. Lastly, the imputed data is passed to the chosen estimator, and the train/test performance is evaluated.


In [ ]:
#load pipeline
from sklearn.pipeline import Pipeline

#load some preprocessing steps 
#Note earlier we identified the matrixs as having a low sparsity ratio 0.3 so will use PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#loading models 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

#load our cross validator
from sklearn.model_selection import GridSearchCV

knn_pipe = Pipeline([('inf_b_gone',RemoveInfs()), ('simp', SimpleImputer()), ('knn', KNeighborsClassifier())])
knn_pipe.fit(X_train, y_train)

tree_pipe = Pipeline([('inf_b_gone',RemoveInfs()), ('simp', SimpleImputer()), ('tree', DecisionTreeClassifier())])
tree_pipe.fit(X_train, y_train)

#How did these preforme on our train and test sets
print('|**********KNN**********|\n', 
      'Best Score Train: \n', knn_pipe.score(X_train, y_train), '\n'
      'Best Score: \n', knn_pipe.score(X_test, y_test), '\n',)

print('|**********Tree**********|\n', 
      'Best Score Train: \n', tree_pipe.score(X_train, y_train), '\n'
      'Best Score: \n', tree_pipe.score(X_test, y_test), '\n',)

|**********KNN**********|
 Best Score Train: 
 0.9081481481481481 
Best Score: 
 0.8911917098445595 

|**********Tree**********|
 Best Score Train: 
 1.0 
Best Score: 
 0.9792746113989638 



### Observations on Results

**K-Nearest Neighbors (KNN):** The training performance is similar to the test performance. It is likely that scaling the features could further improve the model's performance.

**Decision Tree:** The training score is perfect, while the test score is 2% lower, which suggests potential overfitting. Consider tuning the model to mitigate overfitting.


## Robust evaluation

In this section, we are interested in more rigorous techniques by implementing more sophisticated methods, for instance:
* Hold-out and cross-validation.
* Hyper-parameter tuning.
* Feature reduction.
* Feature normalisation.



### Enhanced Pipeline Explanation

The original pipeline has been extended with the addition of a Standard Scaler and PCA, chosen for the following reasons:

1. **Standard Scaler:** Ensures that no feature has a bias due to scale of values. This is particularly important for KNN, as this type of model is sensitive to non-scaled values.
2. **PCA:** Given that the feature set is not overly sparse and there are 328 features compared to 2000 cases, some form of feature reduction is necessary to address the curse of dimensionality.

*Note:* For PCA, it is assumed that the variables are linearly related, orthogonal, and that variance is a measure of the 'informativeness' of a variable. Additionally, the choice of imputation is somewhat arbitrary. In practice, domain experts should be consulted for advice on handling missing values in the dataset.


### Parameter Selection for Cross-Validation

#### Preprocessing
- The imputer is set to use the 'mean' strategy. Although this is the default value, the line of code is included for clarity.
- The PCA searches over a range of options for the number of components to retain. The 'mle' option utilizes Minka's MLE to estimate the optimal number of components. Additionally, 20, 50, and 100 components are considered as they provide a sufficiently small ratio of samples to features (at least 10 samples per feature).

#### K-Nearest Neighbors (KNN)
- The search for the optimal number of neighbors includes odd numbers up to 11 to prevent ties during voting.

#### Decision Tree
- Both Gini impurity and entropy are evaluated as criteria for splitting nodes, exploring all possibilities in this regard.
- Powers of 2 are considered for the maximum tree depth (2^4, 2^5, 2^6). Limiting tree depth helps to prevent overfitting and improve generalization, as deeper trees tend to overfit and generalize less effectively.


In [ ]:
#Lets rebuild our piplines adding a standardscaler and some feature reduction
#Note earlier we identified the matrixs as having a low sparsity ratio 0.3
#so I will use PCA

knn_pipe = Pipeline([('inf_b_gone',RemoveInfs()), ('simp', SimpleImputer()), 
                     ('scaler', StandardScaler()), ('pca', PCA()), ('knn', KNeighborsClassifier())])
knn_pipe.fit(X_train, y_train)

tree_pipe = Pipeline([('inf_b_gone',RemoveInfs()), ('simp', SimpleImputer()), 
                      ('scaler', StandardScaler()), ('pca', PCA()), ('tree', DecisionTreeClassifier())])
tree_pipe.fit(X_train, y_train)


#************* KNN *************#
knn_para = {'simp__strategy':['mean'],
            'pca__n_components': ['mle', 20, 50, 100],
            'knn__n_neighbors':[1,3,5,7,11],
            }

knn_gs = GridSearchCV(knn_pipe, knn_para, scoring="accuracy", n_jobs=-1, verbose=10)
knn_gs.fit(X_train, y_train)
print('|**********KNN**********|\n', 
      'Best Params: \n', knn_gs.best_params_, '\n',
      'Best Score: \n', knn_gs.best_score_, '\n',)

#************* Tree *************#
tree_para = {'simp__strategy':['mean'],
             'pca__n_components': ['mle', 20, 50, 100],
             'tree__criterion':['gini','entropy'],
             'tree__max_depth':[16,32,64],
             }

tree_gs = GridSearchCV(tree_pipe, tree_para, scoring="accuracy", n_jobs=-1, verbose=10)
tree_gs.fit(X_train, y_train)
print('|**********Tree**********|\n', 
      'Best Params: \n', tree_gs.best_params_, '\n',
      'Best Score: \n', tree_gs.best_score_, '\n',)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
|**********KNN**********|
 Best Params: 
 {'knn__n_neighbors': 1, 'pca__n_components': 50, 'simp__strategy': 'mean'} 
 Best Score: 
 0.922962962962963 

Fitting 5 folds for each of 24 candidates, totalling 120 fits
|**********Tree**********|
 Best Params: 
 {'pca__n_components': 50, 'simp__strategy': 'mean', 'tree__criterion': 'entropy', 'tree__max_depth': 64} 
 Best Score: 
 0.9007407407407408 



In [ ]:
#How did these preforme on our holdout test set
print('|**********KNN**********|\n', 
      'Best Score: \n', knn_gs.best_estimator_.score(X_test, y_test), '\n',)

print('|**********Tree**********|\n', 
      'Best Score: \n', tree_gs.best_estimator_.score(X_test, y_test), '\n',)


|**********KNN**********|
 Best Score: 
 0.9015544041450777 

|**********Tree**********|
 Best Score: 
 0.8652849740932642 



### Observations on Results

**K-Nearest Neighbors (KNN):** Scaling the features resulted in a minor improvement in the model's performance.

**Decision Tree:** The model's performance decreased, and it is notable that the cross-validation selected the largest available depth. It may be worthwhile to consider increasing the maximum depth, but caution is necessary to avoid overfitting the data.


## New classifier

Replicate the previous task for another classifier. Briefly describe your choice.
Try to create the best model for the given dataset.
Save your best model into your github. And create a single code cell that loads it and evaluate it on the following test dataset:
https://github.com/andvise/DataAnalyticsDatasets/blob/main/dm_assignment2/sat_dataset_test.csv

### Selection of New Classifiers

Two additional classifiers have been chosen for exploration: the Random Forest Classifier and the XGBClassifier, a popular extreme gradient boosting classifier. Both classifiers utilize decision trees but construct them in distinct ways.

#### Random Forest
A random forest consists of multiple decision trees, each grown independently from one another. The final classification is determined by the majority vote of all the trees in the ensemble. This approach offers improved generalization compared to individual decision trees, as it benefits from the collective wisdom of multiple trees.

#### Extreme Gradient Boosting (XGBoost)
Gradient boosting, like random forests, also involves an ensemble of decision trees. However, these trees are grown sequentially, with each tree learning from the previous tree to reduce the error via gradient descent along a loss curve. XGBoost incorporates additional optimizations, such as pruning, regularization, and parallel processing, to enhance the model's performance while reducing overfitting and bias.


### Cross-Validation Parameter Selection

*Important note:* The GridSearchCV with the chosen parameters requires a significant amount of time due to the number of fits involved. To obtain similar results in less time, consider using RandomizedSearchCV instead.

#### Random Forest
- **n_estimators:** The number of trees in the forest. Increasing the number of trees generally leads to better performance.
- **Criterion:** Both Gini impurity and entropy are evaluated as splitting criteria to identify the best option.
- **max_depth:** Maximum tree depth is explored, with a preference for smaller trees to prevent overfitting.

#### XGBoost
The following parameter sets are searched for XGBoost:

- **n_estimators:** The number of trees in the forest. Increasing the number of trees generally leads to better performance.
- **eta:** This is the learning rate of the gradient descent. A balanced learning rate is desired to avoid overshooting or undershooting the local minima in the loss curve.
- **max_depth:** Maximum tree depth is explored, with a preference for smaller trees to prevent overfitting.

*Final note:* No changes were made to the preprocessing or parameter search for these classifiers.


In [ ]:
# YOUR CODE HERE
rf_pipe = Pipeline([('inf_b_gone',RemoveInfs()), ('simp', SimpleImputer()), 
                    ('scaler', StandardScaler()), ('pca', PCA()), ('rf', RandomForestClassifier())])

rf_para = {'simp__strategy':['mean'],
           'pca__n_components': ['mle', 20, 50, 100],
           'rf__n_estimators':[500, 1000, 2000],
           'rf__criterion':['gini','entropy'],
           'rf__max_depth':[8, 16, 32, 64],
            }

rf_gs = GridSearchCV(rf_pipe, rf_para, scoring="accuracy", n_jobs=-1, verbose=10)
rf_gs.fit(X_train, y_train)
print('|**********Tree**********|\n', 
      'Best Params: \n', rf_gs.best_params_, '\n',
      'Best Score: \n', rf_gs.best_score_, '\n',)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
|**********Tree**********|
 Best Params: 
 {'pca__n_components': 50, 'rf__criterion': 'gini', 'rf__max_depth': 32, 'rf__n_estimators': 1000, 'simp__strategy': 'mean'} 
 Best Score: 
 0.9488888888888889 



In [ ]:
xgbm_pipe = Pipeline([('inf_b_gone',RemoveInfs()), ('simp', SimpleImputer()), 
                    ('scaler', StandardScaler()), ('pca', PCA()), ('xgbm', XGBClassifier())])

xgbm_para = {'simp__strategy':['mean'],
            'pca__n_components': ['mle', 20, 50, 100],
            'xgbm__n_estimators':[500,1000,2000],
            'xgbm__max_depth':[8, 16, 32, 64],
            'xgbm__eta':[0.1,0.3,0.5] #test different learning rates to try to avoid local minima
            }

xgbm_gs = GridSearchCV(xgbm_pipe,xgbm_para, scoring="accuracy", n_jobs=-1, verbose=10)
xgbm_gs.fit(X_train, y_train)

print('|**********XGBM**********|\n', 
      'Best Params: \n', xgbm_gs.best_params_, '\n',
      'Best Score: \n', xgbm_gs.best_score_, '\n',)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
|**********XGBM**********|
 Best Params: 
 {'pca__n_components': 'mle', 'simp__strategy': 'mean', 'xgbm__eta': 0.1, 'xgbm__max_depth': 8, 'xgbm__n_estimators': 2000} 
 Best Score: 
 0.9540740740740741 



In [ ]:
#How did these preforme on our holdout test set
print('|**********Random Forest**********|\n', 
      'Best Score: \n', rf_gs.best_estimator_.score(X_test, y_test), '\n',)

print('|**********XGBM**********|\n', 
      'Best Score: \n', xgbm_gs.best_estimator_.score(X_test, y_test), '\n',)

|**********Random Forest**********|
 Best Score: 
 0.924006908462867 

|**********XGBM**********|
 Best Score: 
 0.9481865284974094 



In [ ]:
df_test = pd.read_csv('https://github.com/andvise/DataAnalyticsDatasets/blob/f4c1e07915ddfe98f0f5434ec3f0e7f3900f35ab/dm_assignment2/sat_dataset_test.csv?raw=True')

X_valid, y_valid = df_test[features], df_test[labels]

print('|**********KNN GS**********|\n', 
      'Best Score: \n', knn_gs.best_estimator_.score(X_valid, y_valid), '\n',)

print('|*********Tree GS**********|\n', 
      'Best Score: \n', tree_gs.best_estimator_.score(X_valid, y_valid), '\n',)

print('|**********Ransom Forest GS**********|\n', 
      'Best Score: \n', rf_gs.best_estimator_.score(X_valid, y_valid), '\n',)

print('|**********XGBM GS**********|\n', 
      'Best Score: \n', xgbm_gs.best_estimator_.score(X_valid, y_valid), '\n',)


|**********KNN GS**********|
 Best Score: 
 0.979296066252588 

|*********Tree GS**********|
 Best Score: 
 0.9606625258799172 

|**********Ransom Forest GS**********|
 Best Score: 
 0.9875776397515528 

|**********XGBM GS**********|
 Best Score: 
 0.9937888198757764 



### Observations on Results

Among the four classifiers explored, XGBoost demonstrated the best performance. As the test and training scores are closely aligned and XGBoost incorporates techniques to reduce overfitting, this model is chosen as the most suitable for the given task.

In [ ]:
from joblib import dump, load
dump(xgbm_gs.best_estimator_, 'bestmodel.joblib')

['bestmodel.joblib']